In [1]:
import os, sys, shutil
from random import random
from glob import glob
from pydub import AudioSegment

In [2]:
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)


for path in glob('data/nlp_egg/*/*'):
    split_path = path.split('\\')
    root, cls, filename = split_path[0], split_path[1], split_path[2]
    sound = AudioSegment.from_file(path, filename.split('.')[-1])
    # sound = sound.set_frame_rate(16000)
    normalized_sound = match_target_amplitude(sound, -20.0)
    normalized_sound_path = os.path.join(root + '_normalized', cls, filename)
    os.makedirs(os.path.dirname(normalized_sound_path), exist_ok=True)
    normalized_sound.export(''.join(normalized_sound_path.split('.')[:-1]) + '.wav', format="wav", parameters=["-sample_fmt", 's16', '-sample_rate', '16000'])

In [13]:
import hashlib

TILtrain = glob('data/TIL_train/*/*.wav')
TILtrain_hashes = {f: hashlib.sha256(open(f, 'rb').read()).hexdigest() for f in TILtrain}
other = glob('data/NCSE_sorted/*.wav')
other = {f: hashlib.sha256(open(f, 'rb').read()).hexdigest() for f in other}
for k, v in TILtrain_hashes.items():
    if v in CREMAD_hashes.values():
        print(k)

In [5]:
for path in glob('data/self_recorded_normalized/*/*'):
    # Convert all files to 16khz
    sound = AudioSegment.from_file(path, path.split('.')[-1])
    sound = sound.set_frame_rate(16000)
    sound_path = path.replace('self_recorded_normalized', 'self_recorded_normalized_16khz')
    os.makedirs(os.path.dirname(sound_path), exist_ok=True)
    sound.export(sound_path, format="wav", parameters=["-sample_fmt", 's16'])

In [ ]:
# Handle torchaudio zero tensor bug

import torch
import torchaudio
import glob
import shutil
import os


for path in glob.glob('data/nlp_egg_normalized/*/*.wav'):
    if not torch.all(torchaudio.load(path)[0][0][0]):
        print(path)
        shutil.move(path, path.replace('nlp_egg_normalized', 'nlp_egg_normalized_torch_bugged'))

In [1]:
import os
import glob
import shutil
import pandas as pd

ans = pd.read_csv('answer.csv', usecols=[0, 1], names=['filename', 'label'])
# ans.columns = ['filename', 'label']
# ans.to_csv('answer.csv', index=False)
for path in glob.glob('data/TIL_test_normalized/*.wav'):
    label = ans[ans['filename'] == path.split(os.path.sep)[-1]]['label'].tolist()[0]
    new_path = os.path.join('data/TIL_ans', label, path.split(os.path.sep)[-1])
    os.makedirs(os.path.join('data/TIL_ans', label), exist_ok=True)
    shutil.copy(path, new_path)